# Highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data

This notebook is primarily an intermediate to introducing to the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers..ipynb). Here the steps to see the difference between complexes represented in hu.MAP 2.0 and hu.MAP 3.0 data is done with one example. You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins. In ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb), it will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data. And that is more likely where you want to invest your time. You may want to quickly san the notebook below though to see the types of reports you'll expect when you give the snakemake file your list of identifiers. And with more details here it may help you troubleshoot such snakemake-generated reports. 

This effort just is meant to highlight possible differences. There is some approximating done to determine possible corresponding complexes that inherently brings in some possible judgement calls. You'll need to explore the results from each for yourself to compare in depth. The hu.MAP 2.0 data can be explored in sessions launched from my [humap2-binder repo](https://github.com/fomightez/humap2-binder).

------------

## Step #1: Preparation

The preparation parallels the notebooks ['Use of hu.MAP 3.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb) and ['Use of hu.MAP 2.0 data programmatically with Python, taking advantage of Jupyter features'](Working_with_hu.MAP3_data_with_Python_in_Jupyter_Basics.ipynb), and so see them for more insight.

Get the data for hu.MAP 2.0 and hu.MAP 3.0:

In [1]:
!curl -OL https://raw.githubusercontent.com/fomightez/humap2-binder/refs/heads/main/additional_nbs/standardizing_initial_data/humap2_complexes_20200809InOrderMatched.csv
!curl -OL https://raw.githubusercontent.com/fomightez/humap3-binder/refs/heads/main/additional_nbs/standardizing_initial_data/hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  502k  100  502k    0     0  1870k      0 --:--:-- --:--:-- --:--:-- 1875k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1243k  100 1243k    0     0  2426k      0 --:--:-- --:--:-- --:--:-- 2424k


Get an accessory scripts for adding information about the proteins in the complexes & collecting differences between two versions:

In [2]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/make_lookup_table_for_extra_info4complexes.py
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/two_comp_three_details_plus_table.ipy
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/look_for_proteins_going_missing.py
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/look_for_majority_complex_member_going_missing.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2893  100  2893    0     0  16392      0 --:--:-- --:--:-- --:--:-- 16344


##### Put the data on the complexes into Pandas dataframe

In [3]:
!curl -OL https://raw.githubusercontent.com/fomightez/structurework/refs/heads/master/humap3-utilities/complexes_rawCSV_to_df.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1007  100  1007    0     0   3547      0 --:--:-- --:--:-- --:--:--  3558


In [1]:
!uv run complexes_rawCSV_to_df.py humap2_complexes_20200809InOrderMatched.csv
import pandas as pd
rd2_df = pd.read_pickle('raw_complexes_pickled_df.pkl')
!uv run complexes_rawCSV_to_df.py hu.MAP3.0_complexes_wConfidenceScores_total15326_wGenenames_20240922InOrderMatched.csv
rd3_df = pd.read_pickle('raw_complexes_pickled_df.pkl')

Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               Reading inline script metadata from `complexes_rawCSV_to_df.py`
⠙                                                                               



--------

## Analyze complexes in hu.MAP 2.0 vs. hu.MAP 3.0 for a protein


In [2]:
search_term = "ROGDI"

Get complexes that protein is in in both sets of data, adding extra information like done in earlier notebooks.  
Then use that to make a summary table for change from version 2.0 to version 3.0.  
(Running  the next cell will do all that, and so be patient as getting the additional information can take several minutes.)

In [3]:
%run -i two_comp_three_details_plus_table.ipy


Correspondences (considering shared complex members) for version 2 vs. 3 with improvement, shared items, and coverage ratio:
   HuMAP2_ID size_df2  rank      HuMAP3_ID  size_df3 improvement_in_v3 shared_items coverage_ratio
HuMAP2_01834        5     1 huMAP3_12042.1        56                51            4           80.0
HuMAP2_03388        4     2 huMAP3_10511.1        35                31            3           75.0
HuMAP2_01148        2     3 huMAP3_13872.1        29                27            2          100.0
                          4 huMAP3_09477.1        26                na           na             na
                          5 huMAP3_09873.1        24                na           na             na
                          6 huMAP3_08678.1        22                na           na             na
                          7 huMAP3_09242.1        20                na           na             na
                          8 huMAP3_07329.1        20                na           n

That above should be a summary table quickly giving a sense of the improvement in 3.0 data. (Or lack thereof? Only if you have substituted with your protein of interest and it goes against the general trend.)

Despite general improvment, I noticed some information present in version 2.0 complexes went missing in complexes reported in version 3.0.
    
#### Check for disappearing proteins in complexes

Compare the corresponding complexes to get a sense if anything disappears between them.  
Also, begin to look out for proteins disappearing entirely.

In [13]:
%run -i look_for_proteins_going_missing.py


Disappearing Items between Hu.MAP2.0 and Hu.MAP3.0 data:

HuMAP2 ID: HuMAP2_01834 -> HuMAP3 ID: huMAP3_12042.1
Total items in HuMAP2: 5
Total items in HuMAP3: 56
Percent of items lost: 20.00%
UniProt ACCs Lost From Corresponding Complex:
  - P50993 (AND NOT IN ANY OTHER Hu.MAP3.0 ROGDI-complexes!!!)

HuMAP2 ID: HuMAP2_03388 -> HuMAP3 ID: huMAP3_10511.1
Total items in HuMAP2: 4
Total items in HuMAP3: 35
Percent of items lost: 25.00%
UniProt ACCs Lost From Corresponding Complex:
  - Q8TDJ6 (But present in other, ROGDI-related Hu.MAP3.0 complexes)


Above reports if something disappears with main focus on the corresponding complexes; however, the next cell will check if a protein obserevd in the majority of complexes with hu.MAP 20 data goes missing in hu.MAP 3.0 data.

In [15]:
%run -i look_for_majority_complex_member_going_missing.py


Hu.MAP2.0-Majority Complex Members Disappearing in Hu.MAP3.0 data:
NONE


 You can change the hard-coded values here to investigate the differences for others. However, chances are you are interested in the differnces for several genes/proteins, and so you'll want to first check out the next notebook in this series, ['Using snakemake to highlight differences between hu.MAP 2.0 and hu.MAP 3.0 data for multiple identifiers'](Using_snakemake_to_highlight_differences_between_hu.MAP2_and_hu.MAP3_data_for_multiple_indentifiers.ipynb). That notebook will work through the steps to run a Snakemake workflow to process the identifiers for many proteins/genes to generate summary reports highlighting differences in human complexes reported in hu.MAP 2.0 vs. hu.MAP 3.0 data.
 
-----

Enjoy!

See my [humap3-binder repo](https://github.com/fomightez/humap3-binder) and [humap3-utilities](https://github.com/fomightez/structurework/humap3-utilities) for related information & resources for this notebook.



-----
